# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-0aebe8cba1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Monarchies") 

Consider the following exploratory information need:

> You investigating existing and past historical monarchies

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P122`    | basic form of government    | predicate |
| `wd:Q38`      | Italy          | node |
| `wd:Q145`     | United Kingdom           | node |
| `wd:Q7269`    | monarchy           | node |




Also consider

```
?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
```

is the BGP to retrieve all **countries that are some instance of a monarchy**

## Workload Goals


1. Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

2. Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

3. Which is or was  the country with the highest population?

4. How many countries are or have been monarchies across continents?

5. Analyze the number of countries that were monarchies
 
   5.1 How many sub-types of monarchy exist?  How many instances of exist of each one?
   
   5.2 Are there cities that have been capital of multiple kingdoms?
   
   5.3 Which are the top-3 monarchies with the earliest inception and still existing?


In [2]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2344')]


1

### Explorative Queries

List some country that are some instance of monarchy with their class name.

In [3]:
queryString="""
SELECT DISTINCT ?p ?pname ?cname WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?pname;
    wdt:P31 ?class.
?class <http://schema.org/name> ?cname.
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q260633'), ('pname', "St. Emmeram's Abbey"), ('cname', 'abbey-principality')]
[('p', 'http://www.wikidata.org/entity/Q260633'), ('pname', "St. Emmeram's Abbey"), ('cname', 'imperial abbey')]
[('p', 'http://www.wikidata.org/entity/Q542165'), ('pname', 'Despotate of Dobruja'), ('cname', 'polity')]
[('p', 'http://www.wikidata.org/entity/Q183150'), ('pname', 'Colchis'), ('cname', 'polity')]
[('p', 'http://www.wikidata.org/entity/Q681120'), ('pname', 'Roman Catholic Diocese of Chur'), ('cname', 'Ecclesiastical circumscription immediately subject to the Holy See')]
[('p', 'http://www.wikidata.org/entity/Q787336'), ('pname', 'Roman Catholic Diocese of Metz'), ('cname', 'Ecclesiastical circumscription immediately subject to the Holy See')]
[('p', 'http://www.wikidata.org/entity/Q150058'), ('pname', 'Travancore'), ('cname', 'princely state')]
[('p', 'http://www.wikidata.org/entity/Q150056'), ('pname', 'Kingdom of Cochin'), ('cname', 'princely state

50

#### United Kingdom

Retrieve relevant predicates and objects connected to the United Kingdom, a well-known monarchy.

In [4]:
queryString="""
SELECT DISTINCT ?p ?pname ?o ?oname WHERE { 

wd:Q145 ?p ?o.
?p <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.
} 
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7867'), ('pname', 'category for maps'), ('o', 'http://www.wikidata.org/entity/Q8607617'), ('oname', 'Category:Maps of the United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P9241'), ('pname', 'demographics of topic'), ('o', 'http://www.wikidata.org/entity/Q1983621'), ('oname', 'demographics of the United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('pname', 'category for people born here'), ('o', 'http://www.wikidata.org/entity/Q7463363'), ('oname', 'Category:Births in the United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used'), ('o', 'http://www.wikidata.org/entity/Q541279'), ('oname', 'Angloromani')]
[('p', 'http://www.wikidata.org/prop/direct/P6'), ('pname', 'head of government'), ('o', 'http://www.wikidata.org/entity/Q180589'), ('oname', 'Boris Johnson')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used'), ('o', 'http://www.wikid

100

Retrieve what United Kingdom is an instance of.

In [5]:
queryString="""
SELECT DISTINCT ?instanceOf ?iname WHERE { 

wd:Q145 wdt:P31 ?instanceOf.
?instanceOf <http://schema.org/name> ?iname.

} 
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('instanceOf', 'http://www.wikidata.org/entity/Q112099'), ('iname', 'island nation')]
[('instanceOf', 'http://www.wikidata.org/entity/Q20181813'), ('iname', 'colonial power')]
[('instanceOf', 'http://www.wikidata.org/entity/Q3624078'), ('iname', 'sovereign state')]
[('instanceOf', 'http://www.wikidata.org/entity/Q6256'), ('iname', 'country')]
[('instanceOf', 'http://www.wikidata.org/entity/Q202686'), ('iname', 'Commonwealth realm')]


5

Check UK's form of government

In [6]:
queryString="""
SELECT DISTINCT ?gov ?gname WHERE { 

wd:Q145 wdt:P122 ?gov.
?gov <http://schema.org/name> ?gname.

} 
"""
print("Results")
run_query(queryString)

Results
[('gov', 'http://www.wikidata.org/entity/Q41614'), ('gname', 'constitutional monarchy')]


1

#### Italy

Retrieve Italy's form of government to see if we only get the current form of governement or a list of previous forms. 

In [7]:
queryString="""
SELECT DISTINCT ?gov ?gname ?instanceOf ?iname WHERE { 

wd:Q38 wdt:P122 ?gov.
?gov <http://schema.org/name> ?gname.
?gov wdt:P31 ?instanceOf.
?instanceOf <http://schema.org/name> ?iname.

} 
"""
print("Results")
run_query(queryString)

Results
[('gov', 'http://www.wikidata.org/entity/Q4198907'), ('gname', 'parliamentary republic'), ('instanceOf', 'http://www.wikidata.org/entity/Q1307214'), ('iname', 'form of government')]


1

It is known that Italy was a monarchy until 1946 therefore I look for the information about the replacement of the monarchy in Italy. Firstly I check if there are other predicates connected to the node Italy that have objects that are instance of 'form of government' (wd:Q1307214) that may resemble past forms of government

In [8]:
queryString="""
SELECT DISTINCT ?p ?pname ?o ?oname WHERE { 

wd:Q38 ?p ?o.
?o wdt:P31 wd:Q1307214.

?p <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.

} 
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q4198907'), ('oname', 'parliamentary republic')]


1

List countries that are some instance of monarchy that could be Italy

In [9]:
queryString="""
SELECT DISTINCT ?p ?pname WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?pname.
FILTER(REGEX(?pname, ".*[I,i]taly.*"))
} 
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q223936'), ('pname', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/entity/Q172579'), ('pname', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/entity/Q3755547'), ('pname', 'Kingdom of Italy')]


3

List predicates associated to the above entities

In [10]:
queryString="""
SELECT DISTINCT ?p ?pname ?o ?oname WHERE{
?italy ?p ?o.
?p <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.

{ SELECT DISTINCT ?italy WHERE { 

?italy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?pname.
FILTER(REGEX(?pname, ".*[I,i]taly.*"))
} } 

}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q3330103'), ('oname', 'parliamentary monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q41614'), ('oname', 'constitutional monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q7269'), ('oname', 'monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q65132216'), ('oname', 'Battle of Cecina')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q548114'), ('oname', 'Free State of Fiume')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q268970'), ('oname', 'Republic of Germa

60

Looks like each of these nodes have a 'replaced by' (wdt:P1366) as predicate. Check it with an ask query where I look if there exist one node where there is no triple ?italy wdt:P1366 ?o. If the answer is false then for each of the node there exists a triple with the predicate replaced by.

In [13]:
queryString="""
ASK WHERE{

FILTER NOT EXISTS{?italy wdt:P1366 ?o.}

{ SELECT DISTINCT ?italy WHERE { 

?italy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?pname.
FILTER(REGEX(?pname, ".*[I,i]taly.*"))
} } 

}
"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

As expected all three entities resembling Italy have the property "replaced by" (wdt:P1366). Just out of curiosity, list the triple with the predicate replaced by.

In [12]:
queryString="""
SELECT DISTINCT ?italy ?iname ?p ?pname ?o ?oname WHERE{
?italy wdt:P1366 ?o.
wdt:P1366 <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.

{ SELECT DISTINCT ?italy ?iname WHERE { 

?italy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?iname.
FILTER(REGEX(?iname, ".*[I,i]taly.*"))
} } 

}
"""
print("Results")
run_query(queryString)

Results
[('italy', 'http://www.wikidata.org/entity/Q3755547'), ('iname', 'Kingdom of Italy'), ('pname', 'replaced by'), ('o', 'http://www.wikidata.org/entity/Q583038'), ('oname', 'Ostrogothic Kingdom')]
[('italy', 'http://www.wikidata.org/entity/Q172579'), ('iname', 'Kingdom of Italy'), ('pname', 'replaced by'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oname', 'Italy')]
[('italy', 'http://www.wikidata.org/entity/Q172579'), ('iname', 'Kingdom of Italy'), ('pname', 'replaced by'), ('o', 'http://www.wikidata.org/entity/Q48742118'), ('oname', 'Kingdom of Italy under Fascism')]


3

List all triples where Kingdom of Italy (wd:Q172579) is the subject to retrieve relevant predicates.

In [14]:
queryString="""
SELECT DISTINCT ?p ?pname ?o ?oname WHERE{

    wd:Q172579 ?p ?o.
    ?p <http://schema.org/name> ?pname.
    ?o <http://schema.org/name> ?oname.
    
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q209857'), ('oname', 'Kingdom of Lombardy–Venetia')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q548114'), ('oname', 'Free State of Fiume')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q42876'), ('oname', 'flag of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q268970'), ('oname', 'Republic of German-Austria')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('pname', 'significant event'), ('o', 'http://www.wikidata.org/entity/Q3922436'), ('oname', 'Proclamation of the Kingdom of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('o', 'http://www.wikidata.org/entity/Q38130'), ('oname', 'League of Nations')]
[('p', 'http:/

43

Some useful IRI retrieved by the query are:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1365`   | replaces   | predicate |
| `wdt:P35`     | head of state   | predicate |
| `wdt:P36`    | capital      | predicate |
| `wdt:P793`     | significant event       | predicate |
| `wdt:P910`     | topic's main category   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P156`   | followed by    | predicate |
| `wdt:P30` | continent | predicate |
| `wd:Q8574257` | Category:Kingdom of Italy (1861–1946) | node |


List predicates of Kingdom of Italy (1861-1946) (wd:Q8574257) to see if we have some interesting information there.

In [15]:
queryString="""
SELECT DISTINCT ?p ?pname ?o ?oname WHERE{

    wd:Q8574257 ?p ?o.
    ?p <http://schema.org/name> ?pname.
    ?o <http://schema.org/name> ?oname.
    
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q172579'), ('oname', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4167836'), ('oname', 'Wikimedia category')]


2

This query retrieved no useful information.

### MONARCHIES REPLACED BY SOME OTHER STATES

From the previous findings I can retrieve states that were monarchies but have been replaced by some other states by filtering the countries that are some instance of monarchy where there exists a triple of the type ?country wdt:P1366 ?obejct. In the following query I list all former monarchies and the relative country that replaced them.

In [16]:
queryString="""
SELECT DISTINCT ?former ?replacedBy WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P1366 ?o.
    ?f <http://schema.org/name> ?former.
    ?o <http://schema.org/name> ?replacedBy.
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('former', 'Jodhpur State'), ('replacedBy', 'India')]
[('former', 'Eastern Hungarian Kingdom'), ('replacedBy', 'Principality of Transylvania')]
[('former', 'East Francia'), ('replacedBy', 'Holy Roman Empire')]
[('former', 'Kingdom of Hawaiʻi'), ('replacedBy', 'Republic of Hawaii')]
[('former', 'Duchy of Luxembourg'), ('replacedBy', 'French First Republic')]
[('former', 'Taifa of Algeciras'), ('replacedBy', 'Taifa of Seville')]
[('former', 'Kotoko kingdom'), ('replacedBy', 'Bornu Empire')]
[('former', 'Province of North Carolina'), ('replacedBy', 'North Carolina')]
[('former', 'Sui dynasty'), ('replacedBy', 'Tang Empire')]
[('former', 'Song dynasty'), ('replacedBy', 'Yuan Empire')]
[('former', 'United Principalities'), ('replacedBy', 'Kingdom of Romania')]
[('former', 'Travancore'), ('replacedBy', 'India')]
[('former', 'Travancore'), ('replacedBy', 'Travancore-Cochin')]
[('former', 'Mataram Sultanate'), ('replacedBy', 'Yogyakarta Sultanate')]
[('former', 'Mataram Sultanate'), (

50

I run a query to count how many former monarchies I retrieved.

In [17]:
queryString="""
SELECT (COUNT(DISTINCT ?f) AS ?replacedMonarchies) WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P1366 ?o.
}
"""
print("Results")
run_query(queryString)

Results
[('replacedMonarchies', '350')]


1

From this result I assess there are 350 monarchies replaced by some other states. Now I count how many replaced monarchies there are for each continent.

In [18]:
queryString="""
SELECT (COUNT(DISTINCT ?f) AS ?monarchies) ?continent WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P30 ?c;
        wdt:P1366 ?o.
    ?f <http://schema.org/name> ?former.
    ?c <http://schema.org/name> ?continent.
}
GROUP BY ?continent
ORDER BY DESC(?monarchies)
"""
print("Results")
run_query(queryString)

Results
[('monarchies', '116'), ('continent', 'Europe')]
[('monarchies', '70'), ('continent', 'Asia')]
[('monarchies', '24'), ('continent', 'Africa')]
[('monarchies', '8'), ('continent', 'North America')]
[('monarchies', '6'), ('continent', 'Insular Oceania')]
[('monarchies', '6'), ('continent', 'Eurasia')]
[('monarchies', '5'), ('continent', 'South America')]
[('monarchies', '4'), ('continent', 'Americas')]
[('monarchies', '1'), ('continent', 'Northern America and Mexico')]
[('monarchies', '1'), ('continent', 'Australia')]


10

### STATISTICAL INFORMATION ABOUT MONARCHIES

#### Population

Now I retrieve some statistical information about monarchies, starting from demographic information. 

In [19]:
queryString="""
SELECT DISTINCT ?p ?pname WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        ?p ?o.
    ?p <http://schema.org/name> ?pname.
    FILTER(regex(?pname, ".*[P,p]opulation.*") || regex(?pname, ".*[D,d]emographic.*"))
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P9241'), ('pname', 'demographics of topic')]


2

I can retrieve the average population of monarchies for each continent by using the property population (wdt:P1082).

In [20]:
queryString="""
SELECT (AVG(?pop) AS ?avgPopulation) ?continent WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P30 ?c;
        wdt:P1082 ?pop.
    ?c <http://schema.org/name> ?continent.
}
GROUP BY ?continent
ORDER BY DESC(?avgPopulation)
"""
print("Results")
run_query(queryString)

Results
[('avgPopulation', '680000000'), ('continent', 'Antarctica')]
[('avgPopulation', '172300000'), ('continent', 'Americas')]
[('avgPopulation', '60683173.714285714285714'), ('continent', 'Eurasia')]
[('avgPopulation', '53700545.557377049180328'), ('continent', 'Asia')]
[('avgPopulation', '50827710'), ('continent', 'Insular Oceania')]
[('avgPopulation', '46301031.730769230769231'), ('continent', 'Africa')]
[('avgPopulation', '17282163'), ('continent', 'Central America')]
[('avgPopulation', '16323124.578947368421053'), ('continent', 'North America')]
[('avgPopulation', '16041879.745562130177515'), ('continent', 'Europe')]
[('avgPopulation', '6500000'), ('continent', 'Northern America and Mexico')]
[('avgPopulation', '181'), ('continent', 'Australia')]


11

Antartica is the continent with the highest average population because it was part of the british commonwealth. Retrieve the monarchy with the highest population and the one with the lowest.

In [21]:
queryString="""
SELECT DISTINCT ?max ?maxPopulation WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        ?m <http://schema.org/name> ?max. 
        FILTER(?pop = ?maxPopulation)
        
        { SELECT (MAX(?pop) AS ?maxPopulation) WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        }    
        }
}
    
"""
print("Results")
run_query(queryString)

Results
[('max', 'British Empire'), ('maxPopulation', '680000000')]


1

In [22]:
queryString="""
SELECT DISTINCT ?min ?minPopulation WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        ?m <http://schema.org/name> ?min. 
        FILTER(?pop = ?minPopulation)
        
        { SELECT (MIN(?pop) AS ?minPopulation) WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        }    
        }
}
    
"""
print("Results")
run_query(queryString)

Results
[('min', 'Principality of Sealand'), ('minPopulation', '4')]


1

I now retrieve the top-10 most populated monarchies.

In [23]:
queryString="""
SELECT DISTINCT ?m ?monarchy ?pop WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        ?m <http://schema.org/name> ?monarchy.

}
ORDER BY DESC(?pop)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q8680'), ('monarchy', 'British Empire'), ('pop', '680000000')]
[('m', 'http://www.wikidata.org/entity/Q8733'), ('monarchy', 'Qing dynasty'), ('pop', '432000000')]
[('m', 'http://www.wikidata.org/entity/Q8733'), ('monarchy', 'Qing dynasty'), ('pop', '400000000')]
[('m', 'http://www.wikidata.org/entity/Q8733'), ('monarchy', 'Qing dynasty'), ('pop', '383100000')]
[('m', 'http://www.wikidata.org/entity/Q34266'), ('monarchy', 'Russian Empire'), ('pop', '181537800')]
[('m', 'http://www.wikidata.org/entity/Q34266'), ('monarchy', 'Russian Empire'), ('pop', '178378800')]
[('m', 'http://www.wikidata.org/entity/Q12557'), ('monarchy', 'Mongol Empire'), ('pop', '160000000')]
[('m', 'http://www.wikidata.org/entity/Q33296'), ('monarchy', 'Mughal Empire'), ('pop', '150000000')]
[('m', 'http://www.wikidata.org/entity/Q17'), ('monarchy', 'Japan'), ('pop', '126434565')]
[('m', 'http://www.wikidata.org/entity/Q34266'), ('monarchy', 'Russian Empire'), ('pop', 

10

#### Inception Date

I now look for information about inception date. I select properties that could indicate the inception date and count how many monarchies have that property. 

In [24]:
queryString="""
SELECT DISTINCT (COUNT(?m) AS ?monarchies) ?p ?pname WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        ?p ?o.
    ?p <http://schema.org/name> ?pname.
    FILTER(regex(?pname, ".*[D,d]ate.*") || regex(?pname, ".*[I,i]nception.*"))
}
GROUP BY ?p ?pname
ORDER BY DESC(?monarchies)
"""
print("Results")
run_query(queryString)

Results
[('monarchies', '1474'), ('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception')]
[('monarchies', '1332'), ('p', 'http://www.wikidata.org/prop/direct/P576'), ('pname', 'dissolved, abolished or demolished date')]
[('monarchies', '64'), ('p', 'http://www.wikidata.org/prop/direct/P726'), ('pname', 'candidate')]
[('monarchies', '22'), ('p', 'http://www.wikidata.org/prop/direct/P7295'), ('pname', 'Gregorian calendar start date')]
[('monarchies', '16'), ('p', 'http://www.wikidata.org/prop/direct/P991'), ('pname', 'successful candidate')]
[('monarchies', '3'), ('p', 'http://www.wikidata.org/prop/direct/P3407'), ('pname', 'Klosterdatenbank ID')]
[('monarchies', '1'), ('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth')]
[('monarchies', '1'), ('p', 'http://www.wikidata.org/prop/direct/P570'), ('pname', 'date of death')]


8

Some useful IRI retrieved by the query are:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P571`   | inception   | predicate |
| `wdt:P576`     | dissolved, abolished or demolished date   | predicate |
| `wdt:P570`     | date of death   | predicate |
| `wdt:P569`     | date of birth   | predicate |
| `wdt:P726`     | candidate   | predicate |
| `wdt:P991`     | successful candidate   | predicate |

By listing wdt:P571 we can retrieve the inception date of each monarchy. List some inception date to see the information format. 

In [25]:
queryString="""
SELECT DISTINCT ?monarchy ?inception WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
    ?m <http://schema.org/name> ?monarchy.
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'Dʿmt'), ('inception', '-979-01-01T00:00:00Z')]
[('monarchy', 'Ahom Kingdom'), ('inception', '1228-01-01T00:00:00Z')]
[('monarchy', 'Jodhpur State'), ('inception', '1250-01-01T00:00:00Z')]
[('monarchy', 'Baku Khanate'), ('inception', '1735-01-01T00:00:00Z')]
[('monarchy', 'Eastern Hungarian Kingdom'), ('inception', '1526-11-21T00:00:00Z')]
[('monarchy', 'Duchy of the Archipelago'), ('inception', '1207-01-01T00:00:00Z')]
[('monarchy', 'Kingdom of Sicily'), ('inception', '1848-03-25T00:00:00Z')]
[('monarchy', 'East Francia'), ('inception', '0843-01-01T00:00:00Z')]
[('monarchy', 'Kingdom of Hawaiʻi'), ('inception', '1795-01-01T00:00:00Z')]
[('monarchy', 'Duchy of Luxembourg'), ('inception', '1353-01-01T00:00:00Z')]


10

I run a query to see if there were more monarchies founded in the 1800s or in the 1900s. First, I select monarchies with inception date in the 19th century to see how to filter dates.

In [28]:
queryString="""
SELECT ?monarchy ?inception WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P571 ?inception.
        
        ?m <http://schema.org/name> ?monarchy.

        FILTER(?inception > "1800-01-01T00:00:00Z"^^xsd:dateTime
                && ?inception < "1899-12-31T23:59:59Z"^^xsd:dateTime)  
}
ORDER BY ?inception
"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'United Kingdom of Great Britain and Ireland'), ('inception', '1801-01-01T00:00:00Z')]
[('monarchy', 'Kingdom of Etruria'), ('inception', '1801-03-21T00:00:00Z')]
[('monarchy', 'Kingdom of Etruria'), ('inception', '1801-03-21T00:00:00Z')]
[('monarchy', 'Kingdom of Chiangmai'), ('inception', '1802-01-01T00:00:00Z')]
[('monarchy', 'Principality of Salm'), ('inception', '1802-10-30T00:00:00Z')]
[('monarchy', 'Electorate of Salzburg'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Principalit of Lübeck'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Electorate of Württemberg'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Faridkot State'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Principality of Nassau-Orange-Fulda'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Principality of Krautheim'), ('inception', '1803-01-01T00:00:00Z')]
[('monarchy', 'Principality of Aschaffenburg'), ('inception', '1803-01-01T00:00:00Z')]
[('mon

179

Then I perform the ask query.

In [26]:
queryString="""
ASK WHERE{

    {
        SELECT (COUNT(?m) AS ?1800monarchies) WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P571 ?inception.

        FILTER(?inception > "1800-01-01T00:00:00Z"^^xsd:dateTime
                && ?inception < "1899-12-31T23:59:59Z"^^xsd:dateTime)  
        }
    }
    {
        SELECT (COUNT(?m) AS ?1900monarchies) WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P571 ?inception.

        FILTER(?inception > "1900-01-01T00:00:00Z"^^xsd:dateTime
                && ?inception < "1999-12-31T23:59:59Z"^^xsd:dateTime)  
        }
    }
    
    FILTER(?1800monarchies > ?1900monarchies)
}
"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

The query returns true therefore there were more monarchies founded in the 19th century than in the 20th. 

Now I retrieve the top-50 earliest monarchies showing only the inception year. Notice that I need to filter out inception date that are set in the future otherwise the function YEAR() returns an error. 

In [27]:
queryString="""
SELECT ?inceptionYear ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
        
    FILTER(datatype(?inception) = xsd:dateTime)
    FILTER(?inception < NOW())
    
    BIND(YEAR(xsd:dateTime(?inception)) AS ?inceptionYear)
    
    ?m <http://schema.org/name> ?monarchy.        
}
ORDER BY ?inceptionYear
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('inceptionYear', '30'), ('m', 'http://www.wikidata.org/entity/Q25979'), ('monarchy', 'Kushan Empire')]
[('inceptionYear', '68'), ('m', 'http://www.wikidata.org/entity/Q867782'), ('monarchy', 'Kingdom of Funan')]
[('inceptionYear', '88'), ('m', 'http://www.wikidata.org/entity/Q10565487'), ('monarchy', 'Chen princedom')]
[('inceptionYear', '170'), ('m', 'http://www.wikidata.org/entity/Q17150787'), ('monarchy', 'Bharshiva Dynasty')]
[('inceptionYear', '192'), ('m', 'http://www.wikidata.org/entity/Q216786'), ('monarchy', 'Champa')]
[('inceptionYear', '200'), ('m', 'http://www.wikidata.org/entity/Q6412538'), ('monarchy', 'Kingdom of Araba')]
[('inceptionYear', '200'), ('m', 'http://www.wikidata.org/entity/Q27907706'), ('monarchy', 'Shule Kingdom')]
[('inceptionYear', '200'), ('m', 'http://www.wikidata.org/entity/Q5326812'), ('monarchy', 'Early history of Kedah')]
[('inceptionYear', '200'), ('m', 'http://www.wikidata.org/entity/Q2320075'), ('monarchy', 'Langkasuka')]
[('inceptionYe

50

By filtering on the datype of the inception date we lose all monarchies founded before Christ because they are stored as negative years values. I run a query to retrieve the earliest monarchies taking this into account.

In [28]:
queryString="""
SELECT ?inception ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
    
    ?m <http://schema.org/name> ?monarchy.        
}
ORDER BY ?inception
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
[('inception', '11584-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q2349630'), ('monarchy', 'Galactic Empire')]
[('inception', '0030-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q25979'), ('monarchy', 'Kushan Empire')]
[('inception', '0068-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q867782'), ('monarchy', 'Kingdom of Funan')]
[('inception', '0088-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q10565487'), ('monarchy', 'Chen princedom')]
[('inception', '0170-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q17150787'), ('monarchy', 'Bharshiva Dynasty')]
[('inception', '0192-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q216786'), ('monarchy', 'Champa')]
[('inception', '0200-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q5326812'), ('monarchy', 'Early history of Kedah')]
[('inception', '0200-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q2320075'), ('monarchy', 'Langkasuka')]
[('inception', '

15

In [29]:
queryString="""
SELECT ?inception ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
    
    ?m <http://schema.org/name> ?monarchy.        
}
ORDER BY DESC(?inception)
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
[('inception', '-146-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q1275158'), ('monarchy', 'Elymais')]
[('inception', 'nodeID://b223225'), ('m', 'http://www.wikidata.org/entity/Q10863765'), ('monarchy', '㠱国')]
[('inception', '-036-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q28370'), ('monarchy', 'Goguryeo')]
[('inception', '-056-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q28456'), ('monarchy', 'Silla')]
[('inception', 'nodeID://b323316'), ('m', 'http://www.wikidata.org/entity/Q1146786'), ('monarchy', 'Señorío of Cuzcatlán')]
[('inception', 'nodeID://b172051'), ('m', 'http://www.wikidata.org/entity/Q11905395'), ('monarchy', 'Amqa')]
[('inception', 'nodeID://b12972'), ('m', 'http://www.wikidata.org/entity/Q5656071'), ('monarchy', 'Accha')]
[('inception', '-014-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q4781804'), ('monarchy', 'Apraca')]
[('inception', '-059-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q173082'), (

15

By means of these two queries I notice date starting with '-' are ordered as the biggest one. In order to run a query that returns the top-15 earliest monarchies I have to use the function STRAFTER() in order to order all dates before Christ correctly. In fact, if we look at the results above without this manipulation 146 BC is considered before 14 BC. By filtering for only inception that starts with '-' we ensure all dates are BC. We order by descending order since I am considering BC dates.   

In [30]:
queryString="""
SELECT DISTINCT ?inceptionYear ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        <http://schema.org/name> ?monarchy.
    
    FILTER(strstarts(str(?inception), '-'))
    BIND(YEAR(xsd:dateTime(strafter(str(?inception), '-'))) AS ?inceptionYear)
           
}
ORDER BY DESC(?inceptionYear)
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
[('inceptionYear', '2499'), ('m', 'http://www.wikidata.org/entity/Q41642'), ('monarchy', 'Phoenicia')]
[('inceptionYear', '2333'), ('m', 'http://www.wikidata.org/entity/Q4461035'), ('monarchy', 'Akkadian empire')]
[('inceptionYear', '2069'), ('m', 'http://www.wikidata.org/entity/Q169705'), ('monarchy', 'Xia dynasty')]
[('inceptionYear', '2003'), ('m', 'http://www.wikidata.org/entity/Q617218'), ('monarchy', 'Yamhad')]
[('inceptionYear', '2000'), ('m', 'http://www.wikidata.org/entity/Q1760729'), ('monarchy', 'Alashiya')]
[('inceptionYear', '1829'), ('m', 'http://www.wikidata.org/entity/Q733897'), ('monarchy', 'First Babylonian Empire')]
[('inceptionYear', '1700'), ('m', 'http://www.wikidata.org/entity/Q183150'), ('monarchy', 'Colchis')]
[('inceptionYear', '1649'), ('m', 'http://www.wikidata.org/entity/Q178665'), ('monarchy', 'Mitanni')]
[('inceptionYear', '1600'), ('m', 'http://www.wikidata.org/entity/Q13053139'), ('monarchy', 'Hittite Empire')]
[('inceptionYear', '1599'), ('m', 

15

By following a similar reasoning I retrieve the most recent founded monarchies.

In [31]:
queryString="""
SELECT DISTINCT ?inceptionYear ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        <http://schema.org/name> ?monarchy.
    
    FILTER(datatype(?inception) = xsd:dateTime)
    BIND(YEAR(xsd:dateTime(?inception)) AS ?inceptionYear)
           
}
ORDER BY DESC(?inceptionYear)
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 22007 Error DT006: Cannot convert 11584-01-01T00:00:00Z to datetime : Incorrect year field length\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-0aebe8cba1-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?inceptionYear ?m ?monarchy WHERE{\n\n    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;\n        wdt:P571 ?inception;\n        <http://schema.org/name> ?monarchy.\n    \n    FILTER(datatype(?inception) = xsd:dateTime)\n    BIND(YEAR(xsd:dateTime(?inception)) AS ?inceptionYear)\n           \n}\nORDER BY DESC(?inceptionYear)\nLIMIT 15\n'


I notice the query throws an error because there is an inception date with year length larger than 4 therefore I filter out inception date in the future for now. 

In [32]:
queryString="""
SELECT DISTINCT ?inceptionYear ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        <http://schema.org/name> ?monarchy.
    
    FILTER(datatype(?inception) = xsd:dateTime && ?inception < NOW())
    BIND(YEAR(xsd:dateTime(?inception)) AS ?inceptionYear)
           
}
ORDER BY DESC(?inceptionYear)
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
[('inceptionYear', '2020'), ('m', 'http://www.wikidata.org/entity/Q105082407'), ('monarchy', 'Kingdom of Africa')]
[('inceptionYear', '2016'), ('m', 'http://www.wikidata.org/entity/Q27304761'), ('monarchy', 'Asgardia')]
[('inceptionYear', '2011'), ('m', 'http://www.wikidata.org/entity/Q7382038'), ('monarchy', 'Romanov Empire')]
[('inceptionYear', '2008'), ('m', 'http://www.wikidata.org/entity/Q551798'), ('monarchy', 'Grand Duchy of Flandrensis')]
[('inceptionYear', '2006'), ('m', 'http://www.wikidata.org/entity/Q2429253'), ('monarchy', 'Islamic State of Iraq and the Levant')]
[('inceptionYear', '2004'), ('m', 'http://www.wikidata.org/entity/Q1083009'), ('monarchy', 'Principality of Wy')]
[('inceptionYear', '2004'), ('m', 'http://www.wikidata.org/entity/Q1423680'), ('monarchy', 'Gay and Lesbian Kingdom of the Coral Sea Islands')]
[('inceptionYear', '1999'), ('m', 'http://www.wikidata.org/entity/Q2429253'), ('monarchy', 'Islamic State of Iraq and the Levant')]
[('inceptionYear', 

15

Since in the previous query I noticed there is a monarchy founded in the future (probably related to fiction) I run a query to retrieve all possible monarchies founded in the future by filtering on inception.

In [33]:
queryString="""
SELECT ?inception ?m ?monarchy WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        <http://schema.org/name> ?monarchy.
    
    FILTER(?inception > NOW() && !strstarts(str(?inception),'-'))
           
}
ORDER BY ?inception
"""
print("Results")
run_query(queryString)

Results
[('inception', 'nodeID://b172051'), ('m', 'http://www.wikidata.org/entity/Q11905395'), ('monarchy', 'Amqa')]
[('inception', 'nodeID://b223225'), ('m', 'http://www.wikidata.org/entity/Q10863765'), ('monarchy', '㠱国')]
[('inception', 'nodeID://b323316'), ('m', 'http://www.wikidata.org/entity/Q1146786'), ('monarchy', 'Señorío of Cuzcatlán')]
[('inception', '11584-01-01T00:00:00Z'), ('m', 'http://www.wikidata.org/entity/Q2349630'), ('monarchy', 'Galactic Empire')]
[('inception', 'nodeID://b12972'), ('m', 'http://www.wikidata.org/entity/Q5656071'), ('monarchy', 'Accha')]


5

I run some queries to understand what "nodeID://..." stands for.

In [34]:
queryString="""
SELECT ?monarchy ?pname ?oname WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        ?p ?o;
        <http://schema.org/name> ?monarchy.
    
    ?p <http://schema.org/name> ?pname.
    ?o <http://schema.org/name> ?oname.
    
    FILTER(contains(str(?inception),'nodeID'))
           
}
ORDER BY ?monarchy
"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'Accha'), ('pname', 'location'), ('oname', 'Morocco')]
[('monarchy', 'Accha'), ('pname', 'instance of'), ('oname', 'principality')]
[('monarchy', 'Amqa'), ('pname', 'instance of'), ('oname', 'principality')]
[('monarchy', 'Amqa'), ('pname', 'time period'), ('oname', 'classical antiquity')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'language used'), ('oname', 'Poqomam')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'official language'), ('oname', 'Pipil')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'country'), ('oname', 'Señorío of Cuzcatlán')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'instance of'), ('oname', 'historical country')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'basic form of government'), ('oname', 'monarchy')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'capital'), ('oname', 'Antiguo Cuscatlán')]
[('monarchy', 'Señorío of Cuzcatlán'), ('pname', 'continent'), ('oname', 'North America')]
[('monarchy', '㠱国'), ('pname

13

In [35]:
queryString="""
SELECT ?monarchy ?inception ?name WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception;
        <http://schema.org/name> ?monarchy.
        
    ?inception <http://schema.org/name> ?name.
    
    FILTER(contains(str(?inception),'nodeID'))
           
}
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [36]:
queryString="""
SELECT ?inception ?p ?pname ?o ?oname WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
        
    ?inception ?p ?o.
    
    ?p <http://schema.org/name> ?pname.
    ?o <http://schema.org/name> ?oname.
    
    FILTER(contains(str(?inception),'nodeID'))
           
}
ORDER BY ?monarchy
"""
print("Results")
run_query(queryString)

Results
Empty


0

After these three queries I can safely say this nodes do not provide any useful information about the inception of the monarchies they refer to.

Finally, we can count how many monarchies were founded per year.

In [37]:
queryString="""
SELECT ?inceptionYear (COUNT(DISTINCT ?m) AS ?monarchies) WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P571 ?inception.
    
    FILTER(datatype(?inception) = xsd:dateTime && ?inception < NOW())
    BIND(YEAR(xsd:dateTime(?inception)) AS ?inceptionYear)
           
}
GROUP BY ?inceptionYear
ORDER BY DESC(?monarchies)
LIMIT 15
"""
print("Results")
run_query(queryString)

Results
[('inceptionYear', '700'), ('monarchies', '27')]
[('inceptionYear', '1806'), ('monarchies', '14')]
[('inceptionYear', '1815'), ('monarchies', '13')]
[('inceptionYear', '1803'), ('monarchies', '12')]
[('inceptionYear', '500'), ('monarchies', '11')]
[('inceptionYear', '1500'), ('monarchies', '9')]
[('inceptionYear', '1180'), ('monarchies', '9')]
[('inceptionYear', '1814'), ('monarchies', '9')]
[('inceptionYear', '1918'), ('monarchies', '9')]
[('inceptionYear', '800'), ('monarchies', '7')]
[('inceptionYear', '1400'), ('monarchies', '7')]
[('inceptionYear', '900'), ('monarchies', '6')]
[('inceptionYear', '1747'), ('monarchies', '6')]
[('inceptionYear', '1867'), ('monarchies', '6')]
[('inceptionYear', '1680'), ('monarchies', '6')]


15

This query shows some interesting results since we can notice in some years (e.g., 700, 1806, 1815) more than 20 monarchies were founded. This result could be caused by some important historical events, for example an indipendence war.

#### Royal Families

We now retrieve information about the dinasty that governs a monarchy. 

In [38]:
queryString="""
SELECT DISTINCT ?monarchy ?p ?pname ?oname WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    <http://schema.org/name> ?monarchy;
    ?p ?o.
    
?p <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.
    
FILTER(regex(?pname, ".*[D,d]inasty.*") || regex(?pname, ".*[F,f]amily.*") || regex(?oname, ".*[D,d]inasty.*"))


} 

"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'Principality of Lippe'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'House of Lippe')]
[('monarchy', 'Mamluk Sultanate'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Bahri dynasty')]
[('monarchy', 'Mamluk Sultanate'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Burji dynasty')]
[('monarchy', 'Principality of Lüneburg'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'House of Welf')]
[('monarchy', 'Kingdom of Pontus'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Mithridatic dynasty')]
[('monarchy', 'Joseon'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'House of Yi')]
[('monarchy', 'Benares State'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Royal House of Benares')]
[('monarchy', 'Duchy of Ferrara'), ('p'

13

I discovered wdt:P53 retrieve the family name of a monarchy. I run a query to see how many monarchies have this property.

In [39]:
queryString="""
SELECT (COUNT(?m) AS ?monarchiesWithFAM) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P53 ?family.
} 

"""
print("Results")
run_query(queryString)

Results
[('monarchiesWithFAM', '17')]


1

Only 17 monarchies have property wdt:P53 therefore I run some queries to retrieve some useful properties that could contain the same information. I recall wdt:P35 indicates the head of state of a country.

In [40]:
queryString="""
SELECT DISTINCT ?monarchy ?head ?p ?pname ?oname WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35 ?h;
    <http://schema.org/name> ?monarchy.
    
?h ?p ?o.
?h <http://schema.org/name> ?head.
?p <http://schema.org/name> ?pname.
?o <http://schema.org/name> ?oname.
FILTER(regex(?pname, ".*[D,d]inasty.*") || regex(?pname, ".*[F,f]amily.*") || regex(?oname, ".*[D,d]inasty.*"))
} 
LIMIT 50

"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'East Francia'), ('head', 'Conrad I of Germany'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Conradines')]
[('monarchy', 'Islands of Refreshment'), ('head', 'Jonathan Lambert'), ('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('oname', 'Lambert')]
[('monarchy', 'Kingdom of Spain'), ('head', 'Amadeo I of Spain'), ('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('oname', 'Savoia')]
[('monarchy', 'Kingdom of Spain'), ('head', 'Amadeo I of Spain'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'House of Savoy')]
[('monarchy', 'Lesotho'), ('head', 'Letsie III of Lesotho'), ('p', 'http://www.wikidata.org/prop/direct/P53'), ('pname', 'family'), ('oname', 'Moshoeshoe I')]
[('monarchy', 'Mongol Empire'), ('head', 'Toghon Temür'), ('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('oname', 'Borjigin')]
[('monarchy', '

50

We can retrieve more information by passing through the head of state family (wdt:P53) or family name (wdt:P734). I can run a query to see if we have more families that run the same kingdom. 

In [41]:
queryString="""
SELECT ?m ?monarchy (COUNT(DISTINCT ?family) AS ?families) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35/(wdt:P53|wdt:P734) ?family;
    <http://schema.org/name> ?monarchy.
    
} 
GROUP BY ?m ?monarchy
ORDER BY DESC(?families)
LIMIT 15

"""
print("Results")
run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q3399982'), ('monarchy', 'Kingdom of Spain'), ('families', '6')]
[('m', 'http://www.wikidata.org/entity/Q147909'), ('monarchy', 'Kingdom of Bulgaria'), ('families', '4')]
[('m', 'http://www.wikidata.org/entity/Q153529'), ('monarchy', 'Duchy of Milan'), ('families', '4')]
[('m', 'http://www.wikidata.org/entity/Q815731'), ('monarchy', 'Principality of Bulgaria'), ('families', '4')]
[('m', 'http://www.wikidata.org/entity/Q650370'), ('monarchy', 'Margraviate of Baden-Durlach'), ('families', '4')]
[('m', 'http://www.wikidata.org/entity/Q34266'), ('monarchy', 'Russian Empire'), ('families', '3')]
[('m', 'http://www.wikidata.org/entity/Q32'), ('monarchy', 'Luxembourg'), ('families', '3')]
[('m', 'http://www.wikidata.org/entity/Q11876909'), ('monarchy', 'State of Brazil'), ('families', '3')]
[('m', 'http://www.wikidata.org/entity/Q398'), ('monarchy', 'Bahrain'), ('families', '3')]
[('m', 'http://www.wikidata.org/entity/Q153080'), ('monarchy', 'Eas

15

I run a query to see  the two families that governed Kingdom of Italy in order to see if the query is counting results in an incorrect way.

In [42]:
queryString="""
SELECT ?monarchy (GROUP_CONCAT(DISTINCT ?family ; separator=', ') AS ?families) WHERE { 

wd:Q172579 wdt:P35/(wdt:P53|wdt:P734) ?f;
    <http://schema.org/name> ?monarchy.
    
?f <http://schema.org/name> ?family.
    
} 
GROUP BY ?monarchy

"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'Kingdom of Italy'), ('families', 'House of Savoy, Savoy')]


1

As I imagined, when we have both properties family and family name we count the same family twice. In order to have correct results I check if we have more head of state with family name information or family information.

In [43]:
queryString="""
ASK WHERE{

    {
        SELECT (COUNT(?f) AS ?fcount) WHERE{
            ?m wdt:P35/wdt:P53 ?f.
        }
    }
    {
        SELECT (COUNT(?fn) AS ?fncount) WHERE{
            ?m wdt:P35/wdt:P734 ?fn.
        }
    }
    
    FILTER(?fcount<?fncount)

}

"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

We have more triple with the family name property than with family property. I now check if for all head of state that have family information (wdt:P53) we also have the family name information (wdt:P734).

In [44]:
queryString="""
ASK WHERE{
    
    ?m wdt:P35 ?h.
    ?h wdt:P53 ?f.
    
    FILTER EXISTS{ ?h wdt:P734 ?fn. }
}

"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

Therefore I can run the same query as before using only the family name information. This time I retrieve all monarchies where more than one family has reigned.

In [45]:
queryString="""
SELECT ?m ?monarchy (COUNT(DISTINCT ?family) AS ?families) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35/wdt:P734 ?family;
    <http://schema.org/name> ?monarchy.
    
} 
GROUP BY ?m ?monarchy
HAVING (COUNT(DISTINCT ?family) > 1) 
ORDER BY DESC(?families)

"""
print("Results")
run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q3399982'), ('monarchy', 'Kingdom of Spain'), ('families', '3')]
[('m', 'http://www.wikidata.org/entity/Q153529'), ('monarchy', 'Duchy of Milan'), ('families', '2')]
[('m', 'http://www.wikidata.org/entity/Q1187978'), ('monarchy', 'Transvaal Colony'), ('families', '2')]
[('m', 'http://www.wikidata.org/entity/Q650370'), ('monarchy', 'Margraviate of Baden-Durlach'), ('families', '2')]
[('m', 'http://www.wikidata.org/entity/Q398'), ('monarchy', 'Bahrain'), ('families', '2')]
[('m', 'http://www.wikidata.org/entity/Q35715'), ('monarchy', 'South Australia'), ('families', '2')]


6

I list all families that reigned in Spain.

In [46]:
queryString="""
SELECT ?monarchy (GROUP_CONCAT(DISTINCT ?family ; separator=', ') AS ?families) WHERE { 

wd:Q3399982 wdt:P35/wdt:P734 ?f;
    <http://schema.org/name> ?monarchy.
    
?f <http://schema.org/name> ?family.
    
} 
GROUP BY ?monarchy

"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'Kingdom of Spain'), ('families', 'Bonaparte, Savoia, Serrano')]


1

I run the previous query by grouping for the country (wdt:P17) instead of the monarchy to see if we have different results due to different names for the same kingdom.

In [47]:
queryString="""
SELECT ?c ?country (COUNT(DISTINCT ?family) AS ?families) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35/wdt:P734 ?family;
    wdt:P17 ?c.
    
?c  <http://schema.org/name> ?country.
    
} 
GROUP BY ?c ?country
HAVING (COUNT(DISTINCT ?family) > 1) 
ORDER BY DESC(?families)

"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q29'), ('country', 'Spain'), ('families', '4')]
[('c', 'http://www.wikidata.org/entity/Q408'), ('country', 'Australia'), ('families', '3')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('families', '2')]
[('c', 'http://www.wikidata.org/entity/Q153529'), ('country', 'Duchy of Milan'), ('families', '2')]
[('c', 'http://www.wikidata.org/entity/Q398'), ('country', 'Bahrain'), ('families', '2')]
[('c', 'http://www.wikidata.org/entity/Q8680'), ('country', 'British Empire'), ('families', '2')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom'), ('families', '2')]


7

Most results match therefore there is no utility in further investigation.

#### Head of State

I now retrieve information about head of states. I retrieve the oldest still alive head of state with the kingdoms he or she reigns. I recall that when looking for inception date I discovered wdt:P569 for date of birth and wdt:P570 for date of death. I add some filter for the birth date in order to avoid displaying head of state that are clearly dead but we miss information about death date and to avoid display information about the fictional kingdom in the future I found earlier.

In [49]:
queryString = """
SELECT ?head ?birth GROUP_CONCAT(DISTINCT ?monarchy ; separator = ', ') AS ?kingdoms WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35 ?h;
    <http://schema.org/name> ?monarchy.

?h <http://schema.org/name> ?head;
    wdt:P569 ?birth.
    
FILTER(datatype(?birth) = xsd:dateTime && ?birth > "1900-01-01T00:00:00Z"^^xsd:dateTime)
FILTER NOT EXISTS{?h wdt:P570 ?death.}   
} 
GROUP BY ?head ?birth
ORDER BY ?birth
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('head', 'Elizabeth II'), ('birth', '1926-04-21T00:00:00Z'), ('kingdoms', 'Antigua and Barbuda, Australia, Barbados, Belize, Canada, Dominion of Fiji, England, Grenada, Jamaica, Jersey, New Zealand, Papua New Guinea, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Saskatchewan, Solomon Islands, South Australia, State of Malta, The Bahamas, Tuvalu, United Kingdom')]
[('head', 'Salman of Saudi Arabia'), ('birth', '1935-12-31T00:00:00Z'), ('kingdoms', 'Saudi Arabia')]
[('head', 'Francis'), ('birth', '1936-12-17T00:00:00Z'), ('kingdoms', 'Vatican City')]
[('head', 'Harald V of Norway'), ('birth', '1937-02-21T00:00:00Z'), ('kingdoms', 'Norway')]
[('head', 'Nawaf Al-Ahmad Al-Jaber Al-Sabah'), ('birth', '1937-06-25T00:00:00Z'), ('kingdoms', 'Kuwait')]
[('head', 'Margrethe II of Denmark'), ('birth', '1940-04-16T00:00:00Z'), ('kingdoms', 'Danish Realm, Denmark, Greenland')]
[('head', 'Konstantinos II of Greece'), ('birth', '1940-06-02T00:00:00Z'), ('kingdoms', 'Ki

10

I run a query to assess the head of state who reigned (or still reigns) over most kingdoms.

In [50]:
queryString = """
SELECT ?head (COUNT(?m) AS ?kingdoms) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P35 ?h.

?h <http://schema.org/name> ?head.

} 
GROUP BY ?head 
ORDER BY DESC(?kingdoms)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('head', 'Elizabeth II'), ('kingdoms', '35')]
[('head', 'Margrethe II of Denmark'), ('kingdoms', '6')]
[('head', 'Abdullah of Pahang'), ('kingdoms', '4')]
[('head', 'Philip III of Spain'), ('kingdoms', '4')]
[('head', 'Charles II of Spain'), ('kingdoms', '3')]
[('head', 'Muhammad bin Saud'), ('kingdoms', '3')]
[('head', 'Monarch of the Kingdom of Asturias'), ('kingdoms', '3')]
[('head', 'Charles V, Holy Roman Emperor'), ('kingdoms', '3')]
[('head', 'Konstantinos II of Greece'), ('kingdoms', '3')]
[('head', 'list of Portuguese monarchs'), ('kingdoms', '3')]
[('head', 'Philip IV of Spain'), ('kingdoms', '3')]
[('head', 'Joseph Bonaparte'), ('kingdoms', '3')]
[('head', 'Willem-Alexander of the Netherlands'), ('kingdoms', '3')]
[('head', 'Philip II of Spain'), ('kingdoms', '3')]
[('head', 'Letsie III of Lesotho'), ('kingdoms', '2')]


15

It turns out Queen Elizabeth II has been head of state of the highest number of countries. 

### COUNTRY WITH THE HIGHEST POPULATION

As we saw before, wdt:P1082 retrieves a country's population. Now I run some queries to retrieve some more demographic information about monarchies.
Earlier we saw that the most populated monarchies is the british empire.

In [51]:
queryString="""
SELECT DISTINCT ?m ?max ?pop WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        ?m <http://schema.org/name> ?max. 
        FILTER(?pop = ?maxPopulation)
        
        { SELECT (MAX(?pop) AS ?maxPopulation) WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
            wdt:P1082 ?pop.
        }    
        }
}
    
"""
print("Results")
run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q8680'), ('max', 'British Empire'), ('pop', '680000000')]


1

I now retrieve some useful information about the British Empire, such as the inception date or if it stilll exists. First I return some properties connected to the British Empire.

In [52]:
queryString="""
SELECT ?p ?pname ?o ?oname WHERE{

        wd:Q8680 ?p ?o.
        ?p <http://schema.org/name> ?pname.
        OPTIONAL { ?o <http://schema.org/name> ?oname.}
}
    
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q83278'), ('oname', 'flag of the United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P237'), ('pname', 'coat of arms'), ('o', 'http://www.wikidata.org/entity/Q165762'), ('oname', 'royal coat of arms of the United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q2078249'), ('oname', 'Britannic Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q1088364'), ('oname', 'Battle of the Lys')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('o', 'http://www.wikidata.org/entity/Q15'), ('oname', 'Africa')]
[('p', 'http://www.wikidata.org/p

69

I run a query to retrieve the inception date.

In [53]:
queryString="""
SELECT ?inception WHERE{

        wd:Q8680 wdt:P571 ?inception.
}
    
"""
print("Results")
run_query(queryString)

Results
[('inception', '1583-01-01T00:00:00Z')]


1

Now I run a query to assess if it has been replaced by some other state.

In [54]:
queryString="""
SELECT ?o ?oname WHERE{

        wd:Q8680 wdt:P1366 ?o.
        ?o <http://schema.org/name> ?oname. 
}
    
"""
print("Results")
run_query(queryString)

Results
Empty


0

We have no information about a state that replaced it. I try to assess if it still exists by selecting the property wdt:P576 that is 'dissolved, abolished or demolished date'.

In [55]:
queryString="""
SELECT ?abolished WHERE{

        wd:Q8680 wdt:P576 ?abolished.
}
    
"""
print("Results")
run_query(queryString)

Results
[('abolished', '1997-01-01T00:00:00Z')]


1

Since I found out the British Empire was abolished in 1997 I can select countries that are in the United Kingdom (wd:Q145) and were founded after 1997, that could have replaced the British Empire. No need to look for all countries because it is common knowledge that the UK are still a monarchy.

In [56]:
queryString="""
SELECT DISTINCT ?m ?monarchy ?inception WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;  
            wdt:P17 wd:Q145;
            wdt:P571 ?inception;
            <http://schema.org/name> ?monarchy.           
            
        FILTER(?inception > "1997-01-01T00:00:00Z"^^xsd:dateTime)
}
    
"""
print("Results")
run_query(queryString)

Results
Empty


0

Since the query returned no useful information I select all monarchies whose country is the United Kingdom (wd:Q145) and their inception date.

In [57]:
queryString="""
SELECT DISTINCT ?m ?monarchy ?inception WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;  
            wdt:P17 wd:Q145;
            wdt:P571 ?inception;
            <http://schema.org/name> ?monarchy.           
}
    
"""
print("Results")
run_query(queryString)

Results
[('m', 'http://www.wikidata.org/entity/Q2578706'), ('monarchy', 'Kingdom of Ceredigion'), ('inception', '0500-01-01T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q2527412'), ('monarchy', 'Rhos'), ('inception', '0401-01-01T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q145'), ('monarchy', 'United Kingdom'), ('inception', '1927-04-12T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q1852345'), ('monarchy', 'Trucial States'), ('inception', '1820-01-08T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q26'), ('monarchy', 'Northern Ireland'), ('inception', '1921-05-08T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q13706'), ('monarchy', 'Principality of Sealand'), ('inception', '1967-09-02T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q8680'), ('monarchy', 'British Empire'), ('inception', '1583-01-01T00:00:00Z')]
[('m', 'http://www.wikidata.org/entity/Q107299'), ('monarchy', 'Kingdom of Northumbria'), ('inception', '0653-01-01T00:00:00Z')]
[('m', 'http://

16

### MONARCHIES ACROSS CONTINENTS

I count the number of monarchies per continent.

In [58]:
queryString = """
SELECT ?continent (COUNT(DISTINCT ?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P30 ?c.

?c <http://schema.org/name> ?continent.

} 
GROUP BY ?continent
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('monarchies', '368')]
[('continent', 'Asia'), ('monarchies', '175')]
[('continent', 'Africa'), ('monarchies', '68')]
[('continent', 'North America'), ('monarchies', '31')]
[('continent', 'Eurasia'), ('monarchies', '14')]
[('continent', 'Insular Oceania'), ('monarchies', '13')]
[('continent', 'Hyboria'), ('monarchies', '13')]
[('continent', 'South America'), ('monarchies', '8')]
[('continent', 'The Continent'), ('monarchies', '7')]
[('continent', 'Thuria'), ('monarchies', '6')]
[('continent', 'Americas'), ('monarchies', '5')]
[('continent', 'Central America'), ('monarchies', '2')]
[('continent', 'Australia'), ('monarchies', '2')]
[('continent', 'Antarctica'), ('monarchies', '1')]
[('continent', 'Northern America and Mexico'), ('monarchies', '1')]
[('continent', 'Caribbean'), ('monarchies', '1')]


16

It comes with no surprise that the continent with most monarchies is Europe. I run a query to count how many monarchies replaced by some other form of state per continent there are in the db.

In [59]:
queryString = """
SELECT ?continent (COUNT(DISTINCT ?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P30 ?c.

?c <http://schema.org/name> ?continent.

FILTER EXISTS{ ?m wdt:P1366 ?o. }

} 
GROUP BY ?continent
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('monarchies', '116')]
[('continent', 'Asia'), ('monarchies', '70')]
[('continent', 'Africa'), ('monarchies', '24')]
[('continent', 'North America'), ('monarchies', '8')]
[('continent', 'Insular Oceania'), ('monarchies', '6')]
[('continent', 'Eurasia'), ('monarchies', '6')]
[('continent', 'South America'), ('monarchies', '5')]
[('continent', 'Americas'), ('monarchies', '4')]
[('continent', 'Northern America and Mexico'), ('monarchies', '1')]
[('continent', 'Australia'), ('monarchies', '1')]


10

Now I count how many still existing monarchies there are per continent. I assume existing monarchies are the one that have been founded after Christ and have no property 'replaced by' (wdt:P1366) or 'dissolved, abolished or demolished date' (wdt:P576) or 'end time' (wdt:P582). _(see the reasoning behind such choice in the former monarchies section)_

In [60]:
queryString = """
SELECT ?continent (COUNT(DISTINCT ?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P571 ?inception;
    wdt:P30 ?c.

?c <http://schema.org/name> ?continent.
FILTER(datatype(?inception) = xsd:dateTime && !strstarts(str(?inception),'-'))

FILTER NOT EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }

} 
GROUP BY ?continent
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('monarchies', '18')]
[('continent', 'North America'), ('monarchies', '16')]
[('continent', 'Asia'), ('monarchies', '13')]
[('continent', 'Insular Oceania'), ('monarchies', '5')]
[('continent', 'Africa'), ('monarchies', '3')]
[('continent', 'Central America'), ('monarchies', '1')]
[('continent', 'Australia'), ('monarchies', '1')]
[('continent', 'Caribbean'), ('monarchies', '1')]


8

I list these monarchies grouped by Continent.

In [61]:
queryString = """
SELECT ?continent GROUP_CONCAT(DISTINCT ?monarchy; separator=', ') AS ?monarchies WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P571 ?inception;
    wdt:P30 ?c;
    <http://schema.org/name> ?monarchy.

?c <http://schema.org/name> ?continent.

FILTER(datatype(?inception) = xsd:dateTime && !strstarts(str(?inception),'-'))

FILTER NOT EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }

} 
GROUP BY ?continent
ORDER BY DESC(COUNT(DISTINCT ?m))
"""

print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('monarchies', 'Andorra, Belgium, County of Tyrol, Danish Realm, Denmark, Jersey, Kingdom of the Netherlands, Liechtenstein, Luxembourg, Monaco, Netherlands, Northern Ireland, Norway, Principality of Sealand, Spain, Sweden, United Kingdom, Vatican City')]
[('continent', 'North America'), ('monarchies', 'Aerican Empire, Antigua and Barbuda, Barbados, Belize, Canada, Danish Realm, Denmark, Greenland, Grenada, Jamaica, Kingdom of the Netherlands, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Saskatchewan, The Bahamas')]
[('continent', 'Asia'), ('monarchies', 'Bahrain, Bhutan, Brunei, Cambodia, Japan, Jordan, Kuwait, Malaysia, Oman, Qatar, Saudi Arabia, Thailand, United Arab Emirates')]
[('continent', 'Insular Oceania'), ('monarchies', 'Australia, New Zealand, Papua New Guinea, Solomon Islands, Tuvalu')]
[('continent', 'Africa'), ('monarchies', 'Eswatini, Kingdom of Africa, Lesotho')]
[('continent', 'Central America'), ('monarchies'

8

### FORMER MONARCHIES

#### Countries that were monarchies

From the results obtained when looking for the inception date of monarchies I notice data also have a property with the abolished date that could indicate the monarchy is no longer present therefore we retrieve some countries with the property wdt:P576.

In [62]:
queryString="""
SELECT DISTINCT ?former ?endDate WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P576 ?endDate.
    ?f <http://schema.org/name> ?former.
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('former', 'Dʿmt'), ('endDate', '-399-01-01T00:00:00Z')]
[('former', 'Duchy of Bouillon'), ('endDate', '1794-01-01T00:00:00Z')]
[('former', 'Ahom Kingdom'), ('endDate', '1826-01-01T00:00:00Z')]
[('former', 'Jodhpur State'), ('endDate', '1949-01-01T00:00:00Z')]
[('former', 'Baku Khanate'), ('endDate', '1806-01-01T00:00:00Z')]
[('former', 'Eastern Hungarian Kingdom'), ('endDate', '1570-08-26T00:00:00Z')]
[('former', 'Duchy of the Archipelago'), ('endDate', '1579-01-01T00:00:00Z')]
[('former', 'Kingdom of Sicily'), ('endDate', '1849-05-15T00:00:00Z')]
[('former', 'East Francia'), ('endDate', '0962-01-01T00:00:00Z')]
[('former', 'Kingdom of Hawaiʻi'), ('endDate', '1893-01-01T00:00:00Z')]
[('former', 'Duchy of Luxembourg'), ('endDate', '1795-01-01T00:00:00Z')]
[('former', 'Alodia'), ('endDate', '1504-01-01T00:00:00Z')]
[('former', 'Duchy of Swabia'), ('endDate', '1313-01-01T00:00:00Z')]
[('former', 'Ghassanids'), ('endDate', '0638-01-01T00:00:00Z')]
[('former', 'Taifa of Algeciras'

50

I count how many monarchies have the above-mentioned property.

In [63]:
queryString="""
SELECT COUNT(?f) AS ?endedMonarchies WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269.
    
    FILTER EXISTS{ ?f wdt:P576 ?endDate. }
}
"""
print("Results")
run_query(queryString)

Results
[('endedMonarchies', '1315')]


1

The same reasoning can be applied to the property 'date of death' (wdt:P570). I select all monarchies that have the just mentioned property to check if the result set provides some useful information about former monarchies.

In [64]:
queryString="""
SELECT DISTINCT ?former ?endDate WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P570 ?endDate.
    ?f <http://schema.org/name> ?former.
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
Empty


0

As expected 'date of death' property provides no useful information in this case. 

From these results I can assess that in order to retrieve all former monarchies I have to select all countries where there exists a triple ?country wdt:P1366 (replaced By) ?o or ?country wdt:P576 (abolished date) ?date. Before running this query I check if there are other properties that could retrieve former monarchies.

In [65]:
queryString="""
SELECT DISTINCT (COUNT(?m) AS ?monarchies) ?p ?pname WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        ?p ?o.
    ?p <http://schema.org/name> ?pname.
    FILTER(regex(?pname, ".*[E,e]nd.*") || regex(?pname, ".*[A,a]bolished.*") || regex(?pname, ".*[A,a]nnex.*"))
}
GROUP BY ?p ?pname
ORDER BY DESC(?monarchies)
"""
print("Results")
run_query(queryString)

Results
[('monarchies', '1332'), ('p', 'http://www.wikidata.org/prop/direct/P576'), ('pname', 'dissolved, abolished or demolished date')]
[('monarchies', '54'), ('p', 'http://www.wikidata.org/prop/direct/P5698'), ('pname', 'Invasive Species Compendium Datasheet ID')]
[('monarchies', '22'), ('p', 'http://www.wikidata.org/prop/direct/P7295'), ('pname', 'Gregorian calendar start date')]
[('monarchies', '18'), ('p', 'http://www.wikidata.org/prop/direct/P582'), ('pname', 'end time')]
[('monarchies', '2'), ('p', 'http://www.wikidata.org/prop/direct/P7284'), ('pname', 'The Independent topic ID')]
[('monarchies', '2'), ('p', 'http://www.wikidata.org/prop/direct/P2402'), ('pname', 'total expenditure')]
[('monarchies', '2'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]


7

Following the reasoning for 'dissolved, abolished or demolished date' (wdt:P576) and 'date of death' (wdt:P570) I check the property 'end time' (wdt:P582).

In [66]:
queryString="""
SELECT DISTINCT ?former ?endDate WHERE{

    ?f (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P582 ?endDate.
    ?f <http://schema.org/name> ?former.
}
"""
print("Results")
run_query(queryString)

Results
[('former', 'Ghassanids'), ('endDate', '0638-01-01T00:00:00Z')]
[('former', 'Roman Empire'), ('endDate', '0395-01-01T00:00:00Z')]
[('former', 'Persian Empire'), ('endDate', '1979-02-11T00:00:00Z')]
[('former', 'Qing dynasty'), ('endDate', '1911-01-01T00:00:00Z')]
[('former', 'Kingdom of Chiangmai'), ('endDate', '1899-01-01T00:00:00Z')]
[('former', 'Habsburg Spain'), ('endDate', '1700-01-01T00:00:00Z')]
[('former', 'Roman Kingdom'), ('endDate', '-508-01-01T00:00:00Z')]
[('former', 'Kenmu Restoration'), ('endDate', '1336-01-01T00:00:00Z')]
[('former', 'Spring and Autumn period'), ('endDate', '-475-01-01T00:00:00Z')]
[('former', 'Chimor'), ('endDate', '1470-01-01T00:00:00Z')]
[('former', 'Chanka people'), ('endDate', '1438-01-01T00:00:00Z')]
[('former', 'Mahameghavahana dynasty'), ('endDate', '0500-01-01T00:00:00Z')]
[('former', 'Gansu Uyghur Kingdom'), ('endDate', '1036-01-01T00:00:00Z')]
[('former', 'Monarchy of Korea'), ('endDate', '1910-08-29T00:00:00Z')]
[('former', 'Hohenloh

15

Now I can retrieve a more complete list of former monarchies by selecting countries where there is a triple of at least one of the relevant properties about former monarchies that are:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1366`   | replaced by  | predicate |
| `wdt:P576`     | dissolved, abolished or demolished date   | predicate |
| `wdt:P582`     | end time   | predicate |

Since we have a large number of former monarchies instead of listing them all I select only the top-50 former monarchies based on the population (wdt:P1082).

In [67]:
queryString="""
SELECT DISTINCT ?monarchy ?pop WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:1366|wdt:P576|wdt:P582 ?o;
        wdt:P1082 ?pop.
    ?m <http://schema.org/name> ?monarchy.
}
ORDER BY DESC(?pop)
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('monarchy', 'British Empire'), ('pop', '680000000')]
[('monarchy', 'Qing dynasty'), ('pop', '432000000')]
[('monarchy', 'Qing dynasty'), ('pop', '400000000')]
[('monarchy', 'Qing dynasty'), ('pop', '383100000')]
[('monarchy', 'Russian Empire'), ('pop', '181537800')]
[('monarchy', 'Russian Empire'), ('pop', '178378800')]
[('monarchy', 'Mongol Empire'), ('pop', '160000000')]
[('monarchy', 'Mughal Empire'), ('pop', '150000000')]
[('monarchy', 'Russian Empire'), ('pop', '125640021')]
[('monarchy', 'Empire of Japan'), ('pop', '97770000')]
[('monarchy', 'Umayyad Caliphate'), ('pop', '70000000')]
[('monarchy', 'German Empire'), ('pop', '67790000')]
[('monarchy', 'German Empire'), ('pop', '64925993')]
[('monarchy', 'Spanish Empire'), ('pop', '60000000')]
[('monarchy', 'Han dynasty'), ('pop', '59594978')]
[('monarchy', 'Han dynasty'), ('pop', '56486856')]
[('monarchy', 'Austria-Hungary'), ('pop', '52800000')]
[('monarchy', 'Austria-Hungary'), ('pop', '51356465')]
[('monarchy', 'German

50

I can also count how many former monarchies there are in the database.

In [68]:
queryString="""
SELECT (COUNT(DISTINCT ?m) AS ?formerMonarchies) WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P1366|wdt:P576|wdt:P582 ?o.
}
"""
print("Results")
run_query(queryString)

Results
[('formerMonarchies', '1124')]


1

Now I retrieved a more complete result as before. Before moving on to inception information I run some relevant queries about monarchies. First, I count how many former monarchies there are accross continents. I recall wdt:P30 assess the continent of a country.

In [69]:
queryString="""
SELECT ?continent (COUNT(DISTINCT ?m) AS ?formerMonarchies) WHERE{

    ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P1366|wdt:P576|wdt:P582 ?o;
        wdt:P30 ?c.
    
    ?c <http://schema.org/name> ?continent.
}
GROUP BY ?continent
ORDER BY DESC(?formerMonarchies)
"""
print("Results")
run_query(queryString)

Results
[('continent', 'Europe'), ('formerMonarchies', '340')]
[('continent', 'Asia'), ('formerMonarchies', '154')]
[('continent', 'Africa'), ('formerMonarchies', '55')]
[('continent', 'North America'), ('formerMonarchies', '15')]
[('continent', 'Eurasia'), ('formerMonarchies', '14')]
[('continent', 'South America'), ('formerMonarchies', '8')]
[('continent', 'Insular Oceania'), ('formerMonarchies', '8')]
[('continent', 'Americas'), ('formerMonarchies', '5')]
[('continent', 'Antarctica'), ('formerMonarchies', '1')]
[('continent', 'Northern America and Mexico'), ('formerMonarchies', '1')]
[('continent', 'Australia'), ('formerMonarchies', '1')]


11

Just out of curiosity, I check if the continent with most former monarchies is also the one with most monarchies today.

In [70]:
queryString="""
ASK WHERE{
    FILTER(?formerContinent = ?continent)
    {
        SELECT (COUNT(DISTINCT ?m) AS ?formerMonarchies) ?formerContinent WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P1366|wdt:P576|wdt:P582 ?o;
        wdt:P30 ?c.
    
        ?c <http://schema.org/name> ?formerContinent.
        }
        GROUP BY ?formerContinent
        ORDER BY DESC(?formerMonarchies)
        LIMIT 1
    }
    {
        SELECT (COUNT(DISTINCT ?m) AS ?monarchies) ?continent WHERE{

        ?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
        wdt:P30 ?c.
    
        ?c <http://schema.org/name> ?continent.
        FILTER NOT EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }
        }
        GROUP BY ?continent
        ORDER BY DESC(?monarchies)
        LIMIT 1
    }
}

"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

#### Monarchy Types

I now retrieve information about monarchy types. I recall wdt:P122 retrieves the basic form of government and wdt:P31 retrieve the property 'instance of'. FIrst I count how many monarchies there are grouped by the same instance of.

In [71]:
queryString = """
SELECT ?i ?instance (COUNT(?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P31 ?i.
?i <http://schema.org/name> ?instance.
} 
GROUP BY ?i ?instance
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('i', 'http://www.wikidata.org/entity/Q3024240'), ('instance', 'historical country'), ('monarchies', '775')]
[('i', 'http://www.wikidata.org/entity/Q208500'), ('instance', 'principality'), ('monarchies', '753')]
[('i', 'http://www.wikidata.org/entity/Q417175'), ('instance', 'kingdom'), ('monarchies', '249')]
[('i', 'http://www.wikidata.org/entity/Q154547'), ('instance', 'duchy'), ('monarchies', '247')]
[('i', 'http://www.wikidata.org/entity/Q3624078'), ('instance', 'sovereign state'), ('monarchies', '74')]
[('i', 'http://www.wikidata.org/entity/Q26830017'), ('instance', 'state in the Holy Roman Empire'), ('monarchies', '74')]
[('i', 'http://www.wikidata.org/entity/Q6256'), ('instance', 'country'), ('monarchies', '66')]
[('i', 'http://www.wikidata.org/entity/Q3403564'), ('instance', 'ecclesiastical principality'), ('monarchies', '65')]
[('i', 'http://www.wikidata.org/entity/Q691981'), ('instance', 'Duchies of Silesia'), ('monarchies', '57')]
[('i', 'http://www.wikidata.org/enti

248

I run the same query grouping by form of government in order to see if there are less different forms of government.

In [72]:
queryString = """
SELECT ?g ?gov (COUNT(?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P122 ?g.
    
?g <http://schema.org/name> ?gov.
    
} 
GROUP BY ?g ?gov
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('g', 'http://www.wikidata.org/entity/Q7269'), ('gov', 'monarchy'), ('monarchies', '509')]
[('g', 'http://www.wikidata.org/entity/Q41614'), ('gov', 'constitutional monarchy'), ('monarchies', '220')]
[('g', 'http://www.wikidata.org/entity/Q184558'), ('gov', 'absolute monarchy'), ('monarchies', '216')]
[('g', 'http://www.wikidata.org/entity/Q584683'), ('gov', 'elective monarchy'), ('monarchies', '37')]
[('g', 'http://www.wikidata.org/entity/Q3330103'), ('gov', 'parliamentary monarchy'), ('monarchies', '37')]
[('g', 'http://www.wikidata.org/entity/Q849242'), ('gov', 'hereditary monarchy'), ('monarchies', '33')]
[('g', 'http://www.wikidata.org/entity/Q4482688'), ('gov', 'feudal monarchy'), ('monarchies', '28')]
[('g', 'http://www.wikidata.org/entity/Q208500'), ('gov', 'principality'), ('monarchies', '24')]
[('g', 'http://www.wikidata.org/entity/Q44405'), ('gov', 'theocracy'), ('monarchies', '19')]
[('g', 'http://www.wikidata.org/entity/Q3208952'), ('gov', 'federal monarchy'), ('mo

63

I now count how many instance of each monarchy there are for still existing monarchies. Still Existing monarchies are retrieved as shown in section _'Monarchies across continents'_.

In [73]:
queryString = """
SELECT ?instanceOf (COUNT(DISTINCT ?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P31 ?i;
    wdt:P571 ?inception.

?i <http://schema.org/name> ?instanceOf.
FILTER(datatype(?inception) = xsd:dateTime && !strstarts(str(?inception),'-') && ?inception<NOW())

FILTER NOT EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }

} 
GROUP BY ?instanceOf
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('instanceOf', 'sovereign state'), ('monarchies', '43')]
[('instanceOf', 'country'), ('monarchies', '37')]
[('instanceOf', 'duchy'), ('monarchies', '16')]
[('instanceOf', 'island nation'), ('monarchies', '15')]
[('instanceOf', 'Commonwealth realm'), ('monarchies', '14')]
[('instanceOf', 'principality'), ('monarchies', '13')]
[('instanceOf', 'micronation'), ('monarchies', '11')]
[('instanceOf', 'constitutional monarchy'), ('monarchies', '10')]
[('instanceOf', 'kingdom'), ('monarchies', '8')]
[('instanceOf', 'emirate of the United Arab Emirates'), ('monarchies', '5')]
[('instanceOf', 'state'), ('monarchies', '5')]
[('instanceOf', 'ecclesiastical principality'), ('monarchies', '4')]
[('instanceOf', 'monarchy'), ('monarchies', '4')]
[('instanceOf', 'colonial power'), ('monarchies', '4')]
[('instanceOf', 'landlocked country'), ('monarchies', '3')]
[('instanceOf', 'dukedom of Spain'), ('monarchies', '3')]
[('instanceOf', 'ducado'), ('monarchies', '3')]
[('instanceOf', 'diocese of th

66

And the same for former monarchies.

In [74]:
queryString = """
SELECT ?instanceOf (COUNT(DISTINCT ?m) AS ?monarchies) WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P31 ?i.

?i <http://schema.org/name> ?instanceOf.

FILTER EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }

} 
GROUP BY ?instanceOf
ORDER BY DESC(?monarchies)
"""

print("Results")
run_query(queryString)

Results
[('instanceOf', 'historical country'), ('monarchies', '626')]
[('instanceOf', 'principality'), ('monarchies', '155')]
[('instanceOf', 'kingdom'), ('monarchies', '132')]
[('instanceOf', 'duchy'), ('monarchies', '111')]
[('instanceOf', 'state in the Holy Roman Empire'), ('monarchies', '60')]
[('instanceOf', 'petty kingdom'), ('monarchies', '47')]
[('instanceOf', 'princely state'), ('monarchies', '44')]
[('instanceOf', 'realm'), ('monarchies', '42')]
[('instanceOf', 'empire'), ('monarchies', '37')]
[('instanceOf', 'ecclesiastical principality'), ('monarchies', '37')]
[('instanceOf', 'dynasty'), ('monarchies', '34')]
[('instanceOf', 'historical period'), ('monarchies', '28')]
[('instanceOf', 'sultanate'), ('monarchies', '27')]
[('instanceOf', 'vassal state'), ('monarchies', '26')]
[('instanceOf', 'state in the Confederation of the Rhine'), ('monarchies', '25')]
[('instanceOf', 'crown land of the Austrian Empire'), ('monarchies', '21')]
[('instanceOf', 'countship'), ('monarchies', '

168

#### Capital of multiple kingdoms

I recall property wdt:P36 retrieve the capital of each country. First I run a query to count the number of kingdoms a city is capital of.

In [75]:
queryString = """
SELECT ?capital (COUNT(DISTINCT ?m) AS ?monarchies) WHERE 
{ 
?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P36 ?c. 
?c <http://schema.org/name> ?capital.

} 
GROUP BY ?capital
HAVING(COUNT(DISTINCT ?m)>1)
ORDER BY DESC(?monarchies)

"""

print("Results")
run_query(queryString)

Results
[('capital', 'London'), ('monarchies', '8')]
[('capital', 'Kutaisi'), ('monarchies', '7')]
[('capital', 'Madrid'), ('monarchies', '7')]
[('capital', 'Rio de Janeiro'), ('monarchies', '7')]
[('capital', 'Vienna'), ('monarchies', '6')]
[('capital', 'Paris'), ('monarchies', '6')]
[('capital', 'Cairo'), ('monarchies', '6')]
[('capital', 'Prague'), ('monarchies', '5')]
[('capital', 'Belgrade'), ('monarchies', '5')]
[('capital', 'Riyadh'), ('monarchies', '5')]
[('capital', 'Berlin'), ('monarchies', '5')]
[('capital', 'Bago'), ('monarchies', '4')]
[('capital', 'Naples'), ('monarchies', '4')]
[('capital', 'Vilnius'), ('monarchies', '4')]
[('capital', 'Palermo'), ('monarchies', '4')]
[('capital', 'Kraków'), ('monarchies', '4')]
[('capital', 'Turin'), ('monarchies', '4')]
[('capital', 'Rome'), ('monarchies', '4')]
[('capital', 'Florence'), ('monarchies', '4')]
[('capital', 'Tbilisi'), ('monarchies', '4')]
[('capital', 'Lahore'), ('monarchies', '4')]
[('capital', 'Copenhagen'), ('monarchi

171

I list for each city all kingdoms it has been capital of.

In [76]:
queryString = """
SELECT ?capital GROUP_CONCAT(DISTINCT ?monarchy; separator=', ') AS ?monarchies WHERE 
{ 
?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P36 ?c;
    <http://schema.org/name> ?monarchy.
    
?c <http://schema.org/name> ?capital.

} 
GROUP BY ?capital
HAVING(COUNT(DISTINCT ?m)>1)
ORDER BY DESC(COUNT(DISTINCT ?m))
"""

print("Results")
run_query(queryString)

Results
[('capital', 'London'), ('monarchies', 'British Empire, England, Great Britain, Kingdom of England, Kingdom of Essex, Kingdom of Wessex, United Kingdom, United Kingdom of Great Britain and Ireland')]
[('capital', 'Madrid'), ('monarchies', 'Crown of Castile, Francoist Spain, Habsburg Spain, Kingdom of Spain, Kingdom of Spain under Joseph Bonaparte, Spain, Spanish Empire')]
[('capital', 'Rio de Janeiro'), ('monarchies', 'Colonial Brazil, Empire of Brazil, Kingdom of Brazil, Kingdom of Portugal, Portuguese Empire, State of Brazil, United Kingdom of Portugal, Brazil and the Algarves')]
[('capital', 'Vienna'), ('monarchies', 'Archduchy of Austria, Austria-Hungary, Austrian Empire, Duchy of Austria, Habsburg Monarchy, Holy Roman Empire')]
[('capital', 'Paris'), ('monarchies', 'Bourbon Restoration, July Monarchy, Kingdom of France, Second French Empire, West Francia')]
[('capital', 'Cairo'), ('monarchies', 'Ayyubid dynasty, Egypt Eyalet, Khedivate of Egypt, Kingdom of Egypt, Mamluk Su

167

#### Top-3 most ancient and still running monarchies

In this query I return the top-3 most ancient and still running monarchies. I can safely assume no monarchy with inception date BC or in the future will be a feasible solution for the question.

In [77]:
queryString = """
SELECT DISTINCT ?monarchy ?inception ?head ?capital WHERE { 

?m (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269;
    wdt:P571 ?inception;
    wdt:P35 ?h;
    wdt:P36 ?cap;
    <http://schema.org/name> ?monarchy.
        
?h <http://schema.org/name> ?head.
?cap <http://schema.org/name> ?capital.

FILTER(datatype(?inception) = xsd:dateTime && !strstarts(str(?inception),'-') && ?inception<NOW())

FILTER NOT EXISTS{ ?m wdt:P1366|wdt:P576|wdt:P582 ?o. }

} 
ORDER BY (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'Sweden'), ('inception', '0700-01-01T00:00:00Z'), ('head', 'Carl XVI Gustaf of Sweden'), ('capital', 'Stockholm')]
[('monarchy', 'Danish Realm'), ('inception', '0800-01-01T00:00:00Z'), ('head', 'Margrethe II of Denmark'), ('capital', 'Copenhagen')]
[('monarchy', 'Denmark'), ('inception', '0800-01-01T00:00:00Z'), ('head', 'Margrethe II of Denmark'), ('capital', 'Copenhagen')]


3